# SEACells Versions

### Imports

In [21]:
import tracemalloc
import time
import scanpy as sc
import pandas as pd

In [22]:
import cupy as cp
import cupyx
import numpy as np
from tqdm import tqdm

from icecream import ic

In [23]:
# from importlib import reload
from SEACells.core import SEACells
# reload(SEACells)

### Initialization

In [24]:
total_memory_bytes = cp.get_default_memory_pool().get_limit()
free_memory_bytes = cp.get_default_memory_pool().free_bytes()

ic(cp.get_default_memory_pool())
ic(cp.get_default_memory_pool().get_limit())

ic(total_memory_bytes)
ic(free_memory_bytes)

ic| cp.get_default_memory_pool(): <cupy.cuda.memory.MemoryPool object at 0x7f5864ed0c10>


ic| cp.get_default_memory_pool().get_limit(): 0
ic| total_memory_bytes: 0
ic| free_memory_bytes: 966144


966144

In [25]:
ad = sc.read("/home/aparna/DATA/aparnakumar/50000_cells/mouse_marioni_50k.h5ad") 
num_cells = 5000
ad = ad[:num_cells]    

In [26]:
def get_data(ad, num_cells, use_gpu, use_sparse): 
  ## User defined parameters

  ## Core parameters 
  # number of SEACells
  n_SEACells = num_cells // 75
  build_kernel_on = 'X_pca' # key in ad.obsm to use for computing metacells
                            # This would be replaced by 'X_svd' for ATAC data

  ## Additional parameters
  n_waypoint_eigs = 10 # Number of eigenvalues to consider when initializing metacells

  model = SEACells(ad, 
                                 use_gpu=use_gpu, 
                                 use_sparse=use_sparse, 
                                 build_kernel_on=build_kernel_on, 
                                 n_SEACells=n_SEACells, 
                                 n_waypoint_eigs=n_waypoint_eigs,
                                 convergence_epsilon = 1e-5)
  model.construct_kernel_matrix()
  model.initialize_archetypes()

  start = time.time()
  tracemalloc.start()

  model.fit(min_iter=10, max_iter=150)

  end = time.time()
  tot_time = end - start

  mem = tracemalloc.get_traced_memory()
  tracemalloc.stop()

  assignments = model.get_hard_assignments()

  # Get the final A and B matrices 
  A = model.A_
  B = model.B_

  # Get the sparsity dataframe
  sparsity = model.sparsity_ratios
  
  return assignments, tot_time, mem, A, B, sparsity

In [27]:
def all_versions(ad):
    # assignments1, time1, mem1 = (0,0,[0, 0]) #get_data(ad, num_cells = num_cells, use_gpu = False, use_sparse=False)
    # assignments2, time2, mem2 = (0, 0, [0, 0])#get_data(ad, num_cells = num_cells, use_gpu = False, use_sparse= True)
    # assignments3, time3, mem3 = (0, 0, [0, 0])#get_data(ad, num_cells = num_cells, use_gpu = True, use_sparse=False)
    # # assignments3, time3, mem3 = get_data(ad, num_cells = num_cells, use_gpu = True, use_sparse=False)
    # assignments4, time4, mem4 = get_data(ad, num_cells = num_cells, use_gpu = True, use_sparse= True)

    # # Write the assignments
    # assignments = [assignments1, assignments2, assignments3, assignments4] 
    
    # # Write the time and memory data
    # comparisons = pd.DataFrame({'version': ['v1: no GPU, no sparse', 'v2:  no GPU, yes sparse', 'v3: yes GPU, no sparse', 'v4: yes GPU, yes sparse'], 
    #                        'time (s)': [time1, time2, time3, time4],
    #                        'peak memory': [mem1[1], mem2[1], mem3[1], mem4[1]]})
    
    # return assignments, comparisons

    assignments4, time4, mem4, A, B, sparsity = get_data(ad, num_cells = num_cells, use_gpu=True, use_sparse=True)
    # If successful, write the time and memory a file "{num_cells}_cells/v4_{timestamp}.txt"
    # Get the timestamp as a number 
    timestamp = time.time()

    # Write the time and memory data 
    with open(f"results2/{num_cells}_cells/v4_{timestamp}.txt", "w") as f: 
        f.write(f"Time: {time4}\n")
        f.write(f"Memory: {mem4}\n")

    # Write the A and B matrices 
    np.save(f"results2/{num_cells}_cells/A_v4_{timestamp}.npy", A)
    np.save(f"results2/{num_cells}_cells/B_v4_{timestamp}.npy", B)
    
    # Write the sparsity dataframe 
    sparsity.to_csv(f"results2/{num_cells}_cells/sparsity_v4_{timestamp}.csv")

    # Clear the GPU memory
    cp.get_default_memory_pool().free_all_blocks()

    assignments2, time2, mem2, A, B, sparsity = get_data(ad, num_cells = num_cells, use_gpu=False, use_sparse=True)
    # If successful, write the time and memory a file "{num_cells}_cells/v4_{timestamp}.txt"
    # Get the timestamp as a number 

    timestamp = time.time()

    # Write the time and memory data 
    with open(f"results2/{num_cells}_cells/v2_{timestamp}.txt", "w") as f: 
        f.write(f"Time: {time2}\n")
        f.write(f"Memory: {mem2}\n")

    # Write the A and B matrixes 
    np.save(f"results2/{num_cells}_cells/A_v2_{timestamp}.npy", A)
    np.save(f"results2/{num_cells}_cells/B_v2_{timestamp}.npy", B)

    # Write the sparsity dataframe 
    sparsity.to_csv(f"results2/{num_cells}_cells/sparsity_v2_{timestamp}.csv")

In [28]:
all_versions(ad)

SPARSE AND GPU
TRYING SEACellsGPU
Welcome to SEACells GPU!
build_graph.SEACellGraph completed
Computing kNN graph using scanpy NN ...
Computing radius for adaptive bandwidth kernel...



Making graph symmetric...
Parameter graph_construction = union being used to build KNN graph...
Computing RBF kernel...



Building similarity LIL matrix...



Constructing CSR matrix...
Building kernel on X_pca
Computing diffusion components from X_pca for waypoint initialization ... 
Determing nearest neighbor graph...


  0%|          | 0/18 [00:00<?, ?it/s]/home/aparna/miniconda/envs/env_SEACells/lib/python3.8/site-packages/cupyx/scipy/sparse/_compressed.py:545: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive.
  warnings.warn('Changing the sparsity structure of a '
 17%|█▋        | 3/18 [00:00<00:00, 26.77it/s]

...

Starting iteration 40.


/data/aparna/aparnakumar/github clone/SEACells/SEACells/cpu.py:524: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.sparsity_ratios = self.sparsity_ratios.append({"A": A.nnz / (A.shape[0]*A.shape[1]),


Completed iteration 40.


/data/aparna/aparnakumar/github clone/SEACells/SEACells/cpu.py:524: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.sparsity_ratios = self.sparsity_ratios.append({"A": A.nnz / (A.shape[0]*A.shape[1]),
/data/aparna/aparnakumar/github clone/SEACells/SEACells/cpu.py:524: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.sparsity_ratios = self.sparsity_ratios.append({"A": A.nnz / (A.shape[0]*A.shape[1]),
/data/aparna/aparnakumar/github clone/SEACells/SEACells/cpu.py:524: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.sparsity_ratios = self.sparsity_ratios.append({"A": A.nnz / (A.shape[0]*A.shape[1]),
/data/aparna/aparnakumar/github clone/SEACells/SEACells/cpu.py:524: FutureWarning: The frame.append method is deprecated 

Converged after 46 iterations.


In [29]:
# mempool = cp.get_default_memory_pool()
# pinned_mempool = cp.get_default_pinned_memory_pool()

# print(mempool.used_bytes())              
# print(mempool.total_bytes())             
# print(pinned_mempool.n_free_blocks())    

In [30]:
# a_cpu = np.ndarray(100, dtype=np.float32)
# a = cp.array(a_cpu)
# print(a.nbytes)                          # 400
# print(mempool.used_bytes())              # 512
# print(mempool.total_bytes())             # 512
# print(pinned_mempool.n_free_blocks())    # 1

In [31]:
# B3 = cp.random.random((13, 1000))
# print(B3.nbytes)                          
# print(mempool.used_bytes())              
# print(mempool.total_bytes())             
# print(pinned_mempool.n_free_blocks())    


In [32]:
# B3 = cp.sparse.csr_matrix(B3)                        
# print(mempool.used_bytes())              
# print(mempool.total_bytes())             
# print(pinned_mempool.n_free_blocks())    

# B3 =B3.argmin(axis=0)
# print(B3.nbytes)                          
# print(mempool.used_bytes())              
# print(mempool.total_bytes())             
# print(pinned_mempool.n_free_blocks())    

In [33]:
# import scipy.sparse

# ATA = cp.random.random((1000, 1000))
# ATA = cp.sparse.csr_matrix(ATA)


# f = ((ATA.multiply(ATA)).sum(axis=0)).ravel()
# g = ATA.diagonal().ravel()

# k =  13
# n = 1000
# d = cp.sparse.csr_matrix((k, n), dtype=cp.float64)
# omega = cp.sparse.csr_matrix((k, n), dtype=cp.float64)

# # keep track of selected indices
# centers = cp.sparse.csr_matrix((k, n), dtype=cp.float64)

# # sampling
# for j in tqdm(range(k)):
#         score = f / g
#         p = score.argmax()
        
#         delta_term1 = ATA[:, p].toarray().squeeze()
#         #     # print(delta_term1)
#         ic(omega[:, p].shape)
#         ic(type(omega[:, p].reshape(-1, 1).multiply(omega)))
#         delta_term2 = omega[:, p].reshape(-1, 1).multiply(omega).sum(axis=0).squeeze()
#         delta = delta_term1 - delta_term2

#         #     # some weird rounding errors
#         # BOOKMARK
#         #delta[p] = [0, delta[p]].max()

#         #     o = delta / np.max([np.sqrt(delta[p]), 1e-6])
#         #     omega_square_norm = np.linalg.norm(o) ** 2
#         #     omega_hadamard = np.multiply(o, o)
#         #     term1 = omega_square_norm * omega_hadamard

#         #     # update f (term2)
#         #     pl = np.zeros(n)
#         #     for r in range(j):
#         #         omega_r = omega[r, :]
#         #         pl += np.dot(omega_r, o) * omega_r

#         #     ATAo = (ATA @ o.reshape(-1, 1)).ravel()
#         #     term2 = np.multiply(o, ATAo - pl)

#         #     # update f
#         #     f += -2.0 * term2 + term1

#         #     # update g
#         #     g += omega_hadamard

#         #     # store omega and delta
#         #     d[j, :] = delta
#         #     omega[j, :] = o

#         #     # add index
#         #     centers[j] = int(p)

#         # return centers

In [34]:
# delta_term1.shape

In [35]:
# p = cp.array([121])
# omega = cp.sparse.csr_matrix((14, 1000), dtype=cp.float64)
# thing = omega[:, p].reshape(-1, 1).multiply(omega).sum(axis=0).squeeze()
# ic(thing.shape)
# ic(type(thing))


# type(thing.sum(axis=0))



In [36]:
# n = 1000
# k = 14

# # precompute M.T * M
# # ATA = M.T @ M
# ATA = cupyx.scipy.sparse.csr_matrix((n, n))


# f = cp.array((ATA.multiply(ATA)).sum(axis=0)).ravel()
# g = cp.array(ATA.diagonal()).ravel()

# d = cp.sparse.csr_matrix((k, n))
# omega = cp.sparse.csr_matrix((k, n))

# # keep track of selected indices
# centers = cp.sparse.csr_matrix((k, n))

# # sampling
# for j in tqdm(range(k)):
#     # Compute score, dividing the sparse f by the sparse g
#     score = f / g
            
#     # Compute p, which is the largest score
#     p = cp.argmax(score)

#     # Compute delta_term1 to be the column of ATA at index p
#     delta_term1 = ATA[:, p].toarray().squeeze()

#     # Compute delta_term2 to be the sum of the outer product of omega and itself
#     delta_term2 = (omega[:, p].reshape(-1, 1).multiply(omega).sum(axis=0).squeeze())
#     delta = delta_term1 - delta_term2

#     # some weird rounding errors
#     delta[p] = max([0, delta[p]])

#     o = delta / max([cp.sqrt(delta[p]), 1e-6])
#     omega_square_norm = cp.linalg.norm(o) ** 2
#     omega_hadamard = cp.multiply(o, o)
#     term1 = omega_square_norm * omega_hadamard

#     # update f (term2)
#     pl = cp.zeros(n)
#     for r in range(j):
#         omega_r = omega[r, :]
#         pl += omega_r.dot(o) * omega_r

#     ATAo = (ATA @ o.reshape(-1, 1)).ravel()
#     term2 = o *  (ATAo - pl)

#     # update f
#     f += -2.0 * term2 + term1

#     # update g
#     g += omega_hadamard

#     # store omega and delta
#     d[j, :] = delta
#     omega[j, :] = o

#     # add index
#     centers[j] = int(p)

# centers

In [37]:
# import cupy

# kernel_matrix =  cupyx.scipy.sparse.rand(1000, 1000, density=0.01, format='csr', dtype=np.float64)
# # Make a sparse csr matrix of random data 
# reconstruction = cupyx.scipy.sparse.rand(1000, 1000, density=0.01, format='csr', dtype=np.float64)

# ic(type(kernel_matrix))
# ic(type(reconstruction))



# cp.linalg.norm(kernel_matrix - reconstruction)

In [38]:
# assignments4, time4, mem4 = get_data(ad, num_cells = num_cells, use_gpu = True, use_sparse= True)

In [39]:
# assignments, comparisons = all_versions(ad)

In [40]:
# Display comparisons 
comparisons

NameError: name 'comparisons' is not defined

## Comparisons 

In [ ]:
# ## Core parameters 
# # number of SEACells
# n_SEACells = num_cells // 75
# build_kernel_on = 'X_pca' # key in ad.obsm to use for computing metacells
#                             # This would be replaced by 'X_svd' for ATAC data

# ## Additional parameters
# n_waypoint_eigs = 10
  
# # Initialize
# model1 = SEACells(ad, use_gpu=False, use_sparse=False, build_kernel_on=build_kernel_on, n_SEACells=n_SEACells, n_waypoint_eigs=n_waypoint_eigs, convergence_epsilon = 1e-5)
# model4 = SEACells(ad, use_gpu=True, use_sparse=True, build_kernel_on=build_kernel_on, n_SEACells=n_SEACells, n_waypoint_eigs=n_waypoint_eigs, convergence_epsilon = 1e-5)
# model3 = SEACells(ad, use_gpu=True, use_sparse=False, build_kernel_on=build_kernel_on, n_SEACells=n_SEACells, n_waypoint_eigs=n_waypoint_eigs, convergence_epsilon = 1e-5)

In [ ]:
# model1.construct_kernel_matrix() 

In [ ]:
# K= model1.K 
# kernel_matrix = model1.kernel_matrix 

In [ ]:
# import scipy.sparse
# model3.K = model1.K
# # convert model1.K to the right type (cupyx.scipy.sparse._csr.csr_matrix) 
# model4.K = cupyx.scipy.sparse.csr_matrix(model1.K)

# ic(type(model1.K))
# ic(type(model3.K)) 
# ic(type(model4.K)) 

# ic(model1.K.shape)
# ic(model3.K.shape)
# ic(model4.K.shape)

In [ ]:
# import scipy

# model3.kernel_matrix = model1.kernel_matrix 
# model4.kernel_matrix = cupyx.scipy.sparse.csr_matrix(model1.kernel_matrix)


# ic(type(model1.kernel_matrix))
# ic(type(model3.kernel_matrix)) 
# ic(type(model4.kernel_matrix)) 

# ic(model1.kernel_matrix.shape)
# ic(model3.kernel_matrix.shape)
# ic(model4.kernel_matrix.shape)

In [ ]:
# model1.initialize_archetypes()

In [ ]:
# model3.archetypes = model1.archetypes 
# model4.archetypes = cp.array(model1.archetypes)

# ic(model1.archetypes.shape)
# ic(model3.archetypes.shape)
# ic(model4.archetypes.shape)

# ic(type(model1.archetypes))
# ic(type(model3.archetypes))
# ic(type(model4.archetypes))


In [ ]:
# model1.initialize()
# ic(model1.archetypes.shape) 
# ic(type(model1.archetypes)) 

# ic(model1.k)
# ic(type(model1.k))

# ic(model1.B0.shape)
# ic(type(model1.B0))

# ic(model1.A0.shape)
# ic(type(model1.A0))

# ic(model1.A_.shape)
# ic(type(model1.A_)) 

# ic(model1.B_.shape)
# ic(type(model1.B_))

# ic(model1.convergence_threshold)
# ic(type(model1.convergence_threshold))

In [ ]:
# model3.initialize()
# ic(model3.archetypes.shape)
# ic(type(model3.archetypes))

# ic(model3.k)
# ic(type(model3.k))

# ic(model3.B0.shape)
# ic(type(model3.B0))

# ic(model3.A0.shape)
# ic(type(model3.A0))

# ic(model3.A_.shape)
# ic(type(model3.A_))

# ic(model3.B_.shape)
# ic(type(model3.B_))

# ic(model3.convergence_threshold)
# ic(type(model3.convergence_threshold))

In [ ]:
# k = len(model4.archetypes)
# cols = cp.arange(k)
# rows = model4.archetypes
# shape = (n, k) 
# data = cp.ones(len(rows))
# B0 = cupyx.scipy.sparse.csr_matrix((cp.ones(len(rows)), (rows, cols)), shape=shape)
# B = B0.copy()



In [ ]:
# model4.initialize() 
# ic(model4.archetypes.shape)
# ic(type(model4.archetypes))

# ic(model4.k)
# ic(type(model4.k))

# ic(model4.B0.shape)
# ic(type(model4.B0))

# ic(model4.A0.shape)
# ic(type(model4.A0))

# ic(model4.A_.shape)
# ic(type(model4.A_))

# ic(model4.B_.shape)
# ic(type(model4.B_))

# ic(model4.convergence_threshold)
# ic(type(model4.convergence_threshold))

In [ ]:
# k = model4.k
# cols = cp.arange(k)
# rows = model4.archetypes
# shape = (n, k)
# B0 = cupyx.scipy.sparse.csr_matrix((cp.ones(len(rows)), (rows, cols)), shape=shape)

In [ ]:
# archetypes_per_cell = int(k * 0.25)
# rows = np.random.randint(0, k, size=(n, archetypes_per_cell)).reshape(-1)
# columns = np.repeat(np.arange(n), archetypes_per_cell)

# ic(rows.shape)
# ic(columns.shape)
# ic(k, n)

# A0 = scipy.sparse.csr_matrix((np.random.random(len(rows)), (rows, columns)), shape=(k, n))


In [ ]:
# archetypes_per_cell = int(k * 0.25)
# rows = np.random.randint(0, k, size=(n, archetypes_per_cell)).reshape(-1)
# columns = np.repeat(np.arange(n), archetypes_per_cell)
#             # print(type(rows))
#             # print(type(columns))

# A0 = scipy.sparse.csr_matrix((np.random.random(len(rows)), (rows, columns)), shape=(k, n))
#             # print(type(A0))
#             # print("STARTING NORMALIZE")
# A0 = cupyx.scipy.sparse.csr_matrix(normalize(A0, axis=0, norm="l1"))


In [ ]:
# # Now we need to compare 

# ic(np.allclose(model1.archetypes, model3.archetypes))
# ic(np.allclose(model1.archetypes, model4.archetypes))

# ic(np.allclose(model1.k, model3.k))
# ic(np.allclose(model1.k, model4.k))

# ic(np.allclose(model1.B0, model3.B0))
# ic(np.allclose(model1.B0, model4.B0.get().todense()))

# ic(np.allclose(model1.A0, model3.A0))
# ic(np.allclose(model1.A0, model4.A0.get().todense()))

# ic(np.allclose(model1.A_, model3.A_))
# ic(np.allclose(model1.A_, model4.A_.get().todense()))

# ic(np.allclose(model1.B_, model3.B_))
# ic(np.allclose(model1.B_, model4.B_.get().todense()))

In [ ]:
# # Compare the A's of model 3 and model 4 
# ic(np.allclose(model3.A0, model4.A0.get().todense()))
# ic(np.allclose(model3.A_, model4.A_.get().todense()))

In [ ]:
# # Set A0 of model 3 and model 4 to be the same as model 1 
# model3.A0 = model1.A0 
# model4.A0 = cupyx.scipy.sparse.csr_matrix(cp.array(model1.A0))

# # check is close 
# ic(np.allclose(model1.A0, model3.A0)) 
# ic(np.allclose(model1.A0, model4.A0.get().todense()))

In [ ]:
# model3.A_ = model3._updateA(model3.B0, model3.A0)
# model4.A_ = model4._updateA(model4.B0, model4.A0)

# # Check is close 
# ic(np.allclose(model1.A_, model3.A_)) 
# ic(np.allclose(model1.A_, model4.A_.get().todense()))

In [ ]:
# # Double check on B_ 

# ic(np.allclose(model1.B_, model3.B_)) 
# ic(np.allclose(model1.B_, model4.B_.get().todense()))

In [ ]:
# RSS3 = model3.compute_RSS(model3.A_, model3.B_)
# RSS4 = model4.compute_RSS(model4.A_, model4.B_)
# RSS1 = model1.compute_RSS(model1.A_, model1.B_)

# ic(type(RSS1)) 
# ic(type(RSS3)) 
# ic(type(RSS4)) 

# ic(RSS1.shape) 
# ic(RSS3.shape) 
# ic(RSS4.shape)

# ic(np.allclose(RSS1, RSS3)) 
# ic(np.allclose(RSS1, RSS4))

In [ ]:
# import scipy.sparse

# reconstruction1 = model1.compute_reconstruction(model1.A_, model1.B_)
# reconstruction4 = model4.compute_reconstruction(cupyx.scipy.sparse.csr_matrix(cp.array(model1.A_)), cupyx.scipy.sparse.csr_matrix(cp.array(model1.B_)))
# reconstruction3 = model3.compute_reconstruction(model3.A_, model3.B_)

# ic(type(reconstruction1)) 
# ic(type(reconstruction3))
# ic(type(reconstruction4)) 

# ic(reconstruction1.shape) 
# ic(reconstruction3.shape)
# ic(reconstruction4.shape)

# ic(np.allclose(reconstruction1, reconstruction3))
# ic(np.allclose(reconstruction1, reconstruction4.get().todense()))

In [ ]:
# import scipy.sparse

# diff1 = model1.kernel_matrix - reconstruction1 
# diff3 = model3.kernel_matrix - reconstruction3
# ic(type(model1.kernel_matrix) )


# diff4 = cupyx.scipy.sparse.csr_matrix(model1.kernel_matrix) - reconstruction4 

# ic(type(diff1)) 
# ic(type(diff3))
# ic(type(diff4)) 

# ic(diff1.shape) 
# ic(diff3.shape )
# ic(diff4.shape) 

# ic(np.allclose(diff1, diff3))
# ic(np.allclose(diff1, diff4.get().todense()))

In [ ]:
# # take the norm of the difference 
# import scipy.sparse.linalg
# from scipy.sparse.linalg import norm


# norm1 = np.linalg.norm(diff1) 
# norm3 = np.linalg.norm(diff3)
# norm4 = np.linalg.norm(diff4.get().todense())
# norm4 = norm(diff4.get())

# ic(type(norm1)) 
# ic(type(norm3)) 
# ic(type(norm4)) 

# ic(norm1)
# ic(norm3)
# ic(norm4)

In [ ]:
# # check A and B 
# ic(model1.A_.shape)
# ic(model3.A_.shape)
# ic(model4.A_.shape)

# ic(model1.B_.shape)
# ic(model3.B_.shape)
# ic(model4.B_.shape)


In [ ]:
# def _updateA(self, B, A_prev):
#         """Compute assignment matrix A using constrained gradient descent via Frank-Wolfe algorithm.

#         Given archetype matrix B and using kernel matrix K, compute assignment matrix A using constrained gradient
#         descent via Frank-Wolfe algorithm.

#         :param B: (n x k csr_matrix) defining SEACells as weighted combinations of cells
#         :param A_prev: (n x k csr_matrix) defining previous weights used for assigning cells to SEACells
#         :return: (n x k csr_matrix) defining updated weights used for assigning cells to SEACells
#         """
#         ic("updateA")
#         n, k = B.shape
#         A = A_prev
#         # ic(type(A))

#         t = 0  # current iteration (determine multiplicative update)

#         # print("bookmark")
#         Ag = A
#         # ic(type(Ag))
#         Bg = B
#         # print(type(self.K))
#         Kg = self.K
#         # ic(type(Bg))
#         # ic(type(Kg))
#         # ic("computed Ag, Bg, Kg")

#         # precompute some gradient terms
#         t2g = Kg.dot(Bg).T
#         t1g = t2g.dot(Bg)
#         # ic("computed t2g, t1g")
#         # ic(type(t2g), type(t1g))

#         # update rows of A for given number of iterations
#         while t < self.max_FW_iter:
#             # compute gradient (must convert matrix to ndarray)
#             # X = t1g.dot(Ag)
#             # print(X.shape)
#             # print(type(X))
#             # print(t2g.shape)
#             # print(type(t2g))
#             # Y = X - t2g 
#             # print(Y.shape)
#             # print(type(Y))
#             # Z = 2.0 *Y 
#             # print(Z.shape)
#             # print(type(Z))
#             # print((cp.subtract(t1g.dot(Ag), t2g)).shape)
#             # print((cp.multiply(2, cp.subtract(t1g.dot(Ag), t2g))).shape)
#             Gg = 2.0*(t1g.dot(Ag) - t2g)
#             # ic(Gg.shape)
#             # ic(type(Gg))
#             # get argmins
#             amins = Gg.argmin(axis = 0)
#             # ic(amins.shape)
#             # ic(type(amins))
#             # loop free implementaton
#             eg = cupyx.scipy.sparse.csr_matrix((k, n), dtype=cp.float64)
#             ic(type(eg))
#             eg[amins, cp.arange(n)] = 1.0
#             ic(type(eg))
#             # print("eg")

#             f = 2.0 / (t + 2.0)
#             ic(type(f))
#             Ag = Ag + (f * (eg - Ag))
#             # Ag = cp.add(Ag, cp.multiply(f, cp.subtract(eg, Ag)))
#             t += 1
#             # print("f, Ag, t")


#         # A = Ag.get()
#         A = Ag


#         del t1g, t2g, Ag, Kg, Gg, Bg, eg, amins
#         cp._default_memory_pool.free_all_blocks()

#         # ic(type(A))
#         return A

In [ ]:
# _updateA(model4, model4.B_, model4.A_)

In [ ]:
# A1 = model1._updateA(model1.B_, model1.A_)
# A3 = model3._updateA(model3.B_, model3.A_)
# A4 = model4._updateA(model4.B_, model4.A_)

In [ ]:
# model1._updateB(A1, model1.B_)
# model3._updateB(A3, model3.B_)
# model4._updateB(A4, model4.B_)

In [ ]:
# model1.step()
# model3.step()
# model4.step()

# model4._updateA(model4.B_, model4.A_) 
# # model4._updateB(model4.A_, model4.B_)


In [ ]:
# B = model4.B_ 
# A_prev = model4.A_ 

# n, k = B.shape
# # print(n, k)
# A = A_prev
# # ic(type(A))

# t = 0  # current iteration (determine multiplicative update)

# # print("bookmark")
# Ag = A
# # ic(type(A))
# Bg = B

# ic(type(model4.K))
# Kg = model4.K